Batch Generate Convex Hulls from Masked Nuclei

Aim : To load, threshold, and detect nuceli for futher analysis.

Updates for v4.2

- forget opencv
- targeted file list
- quick but dirty convex hull detection
- automate file naming to avoid errors
- no plots
- run as a for loop
- hull_3D has a try: except function to ignore empty frames
- relabel hulls as the labels in different frames may not agree (should not redesignate connectivity)

still to do

- prevent overwriting files

In [29]:
##Import modules etc.
import glob
import os
import time
import numpy as np
import mahotas as mh
from skimage import morphology
from skimage.morphology import convex_hull_image
from mahotas.labeled import filter_labeled

In [38]:
#[1] Get targeted filelist
path='C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output/'

target='_Mask5.npy'

filelist=glob.glob(path+str('*')+target)

In [31]:
#Specify output directory
output='ConvexHulls/'
newpath = path+output
if not os.path.exists(newpath):
    os.makedirs(newpath)

**Define Functions**

In [32]:
def labelNuceli(mask, min_vol = 8000):
#By default min_vol = 8000

    labeled_mask, nr_objects=mh.label(mask)
    #print('{} objects were detected.'.format(nr_objects))
    
    real=morphology.remove_small_objects(labeled_mask, min_vol)
    
    #Reset counts  and count the labels, 
    relabeled, n_left = mh.labeled.relabel(real)
    #print('After filtering and relabeling, there are {} nuclei left.'.format(n_left))

    return relabeled

In [33]:
##Define the Object Hulls (~20minutes) 
def hull_3D(labeled_mask):

    filled_mask=np.zeros_like(labeled_mask)
    
    #Iterate through z-slices
    for z, frame in enumerate(labeled_mask):
        #print(z)
        #Could remove on boarders at this point?
        final=np.zeros_like(frame)

        for l in range(frame.max()):
            ll=l+1
            
            try:
                hull = convex_hull_image(frame==ll)*ll
            
            except ValueError:
                hull = np.zeros_like(frame)
            
            final = final + hull

        #filled_mask[z]=final #select line to keep objects on the border
        filled_mask[z], n_hulls = filter_labeled(final, remove_bordering=True)
    return filled_mask

In [34]:
def batchConvHull(filelist):

    ##Loop through each mask and generate labeled convex hulls
    for ff, ffile in enumerate(filelist):
        print ff

        #Get filename
        maskfilename = os.path.basename(filelist[ff])
        #Generate filename for Labeled Convex Hulls
        ConvHullsfilename = maskfilename.replace(target, '_ConvHulls.npy')
        #Generate path for Labeled Convex Hulls
        ConvHullspath = newpath+ConvHullsfilename

        ##Import mask
        mask5 = np.load(filelist[ff])

        #Relabel nuclei
        relabeled = labelNuceli(mask5)

        #Generate labeled convex hulls from 3D stack
        hulls = hull_3D(relabeled)
        
        #Relabel hulls because hulls in different frames may have different numbering
        relabeled = labelNuceli(hulls)

        #Save convex hulls
        np.save(ConvHullspath, relabeled)

In [44]:
[filelist[0],filelist[2]]
#filelist[5:7]

['C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output\\Well02_HeLa_PHD2-IDR-PHD3_1_Mask5.npy',
 'C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output\\Well02_HeLa_PHD2-IDR-PHD3_3_Mask5.npy']

In [46]:
##Time conversion. ~40minutes for 3 3D masks.
t0 = time.time() #remember the brackets
batchConvHull([filelist[0],filelist[2]])
t1 = time.time() #remember the brackets
dt=t1-t0
print (dt)

0
1
1774.921


### END

Trouble shoot Errors below this cell

In [ ]:
##Error
load filename[0]

In [21]:
#For testing
maskpath=filelist[0]
maskpath

'C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output\\Well02_HeLa_PHD2-IDR-PHD3_2_Labeled_Mask5.npy'

In [22]:
maskfilename = os.path.basename(maskpath)
maskfilename

'Well02_HeLa_PHD2-IDR-PHD3_2_Labeled_Mask5.npy'

In [23]:
#Generate filename for Labeled Convex Hulls
ConvHullsfilename = maskfilename.replace(target, '_ConvHulls.npy')
ConvHullsfilename

'Well02_HeLa_PHD2-IDR-PHD3_2_Labeled_ConvHulls.npy'

In [24]:
#Generate path for Labeled Convex Hulls
ConvHullspath = newpath+ConvHullsfilename
ConvHullspath

'C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output/Convex Hulls/Well02_HeLa_PHD2-IDR-PHD3_2_Labeled_ConvHulls.npy'

In [25]:
##Import mask
#mask5=np.load('C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output/Well03_HeLa_PHD2-IDR-PHD3[W1625A]_3_Mask5.npy')
mask5=np.load(maskpath)

In [26]:
relabeled=labelNuceli(mask5)

In [28]:
frame=relabeled[20]

final=np.zeros_like(frame)

for l in range(frame.max()):
    ll=l+1
    print(ll)
    hull = convex_hull_image(frame==ll)*ll
    final = final + hull

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


ValueError: No points given

In [35]:
frame.max()

49

In [33]:
##Error with label 46
nuc45=frame==45
nuc46=frame==46
nuc47=frame==46

In [34]:
nuc45.nonzero()

(array([2000, 2001, 2001, ..., 2047, 2047, 2047], dtype=int64),
 array([1164, 1162, 1164, ..., 1206, 1207, 1208], dtype=int64))

In [32]:
##Time conversion on a labeled stack to labeled convex hulls
t0 = time.time() #remember the brackets
hulls = hull_3D(relabeled)
t1 = time.time() #remember the brackets
dt=t1-t0
#print (dt)
#~20 minutes

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
732.161000013


In [56]:
##Save labeled convex hulls in a new folder.
np.save(ConvHullspath, hulls)